# 1661. Average Time of Process per Machine
<div class="elfjS" data-track-load="description_content"><p>Table: <code>Activity</code></p>

<pre>+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| machine_id     | int     |
| process_id     | int     |
| activity_type  | enum    |
| timestamp      | float   |
+----------------+---------+
The table shows the user activities for a factory website.
(machine_id, process_id, activity_type) is the primary key (combination of columns with unique values) of this table.
machine_id is the ID of a machine.
process_id is the ID of a process running on the machine with ID machine_id.
activity_type is an ENUM (category) of type ('start', 'end').
timestamp is a float representing the current time in seconds.
'start' means the machine starts the process at the given timestamp and 'end' means the machine ends the process at the given timestamp.
The 'start' timestamp will always be before the 'end' timestamp for every (machine_id, process_id) pair.</pre>

<p>&nbsp;</p>

<p>There is a factory website that has several machines each running the <strong>same number of processes</strong>. Write a solution&nbsp;to find the <strong>average time</strong> each machine takes to complete a process.</p>

<p>The time to complete a process is the <code>'end' timestamp</code> minus the <code>'start' timestamp</code>. The average time is calculated by the total time to complete every process on the machine divided by the number of processes that were run.</p>

<p>The resulting table should have the <code>machine_id</code> along with the <strong>average time</strong> as <code>processing_time</code>, which should be <strong>rounded to 3 decimal places</strong>.</p>

<p>Return the result table in <strong>any order</strong>.</p>

<p>The result format is in the following example.</p>

<p>&nbsp;</p>
<p><strong class="example">Example 1:</strong></p>

<pre><strong>Input:</strong> 
Activity table:
+------------+------------+---------------+-----------+
| machine_id | process_id | activity_type | timestamp |
+------------+------------+---------------+-----------+
| 0          | 0          | start         | 0.712     |
| 0          | 0          | end           | 1.520     |
| 0          | 1          | start         | 3.140     |
| 0          | 1          | end           | 4.120     |
| 1          | 0          | start         | 0.550     |
| 1          | 0          | end           | 1.550     |
| 1          | 1          | start         | 0.430     |
| 1          | 1          | end           | 1.420     |
| 2          | 0          | start         | 4.100     |
| 2          | 0          | end           | 4.512     |
| 2          | 1          | start         | 2.500     |
| 2          | 1          | end           | 5.000     |
+------------+------------+---------------+-----------+
<strong>Output:</strong> 
+------------+-----------------+
| machine_id | processing_time |
+------------+-----------------+
| 0          | 0.894           |
| 1          | 0.995           |
| 2          | 1.456           |
+------------+-----------------+
<strong>Explanation:</strong> 
There are 3 machines running 2 processes each.
Machine 0's average time is ((1.520 - 0.712) + (4.120 - 3.140)) / 2 = 0.894
Machine 1's average time is ((1.550 - 0.550) + (1.420 - 0.430)) / 2 = 0.995
Machine 2's average time is ((4.512 - 4.100) + (5.000 - 2.500)) / 2 = 1.456
</pre>
</div>

In [1]:
import os

if os.path.basename(os.getcwd()) != "leetcode":
    os.chdir("..")
    
from src.postgres_manager import PostgresManager

In [2]:
create_table_sql =  """
-- CREATE TYPE activity_type AS ENUM ('start', 'end');
Create table If Not Exists Activity (machine_id int, process_id int, activity_type activity_type, timestamp numeric)
Truncate table Activity
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('0', '0', 'start', '0.712')
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('0', '0', 'end', '1.52')
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('0', '1', 'start', '3.14')
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('0', '1', 'end', '4.12')
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('1', '0', 'start', '0.55')
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('1', '0', 'end', '1.55')
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('1', '1', 'start', '0.43')
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('1', '1', 'end', '1.42')
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('2', '0', 'start', '4.1')
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('2', '0', 'end', '4.512')
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('2', '1', 'start', '2.5')
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('2', '1', 'end', '5')
    """
PM = PostgresManager(user='leetcode',password='leetcode')
PM.check_and_create_database(dbname='leetcode')
PM.drop_table(dbname='leetcode', table_name='activity')
PM.check_and_create_table(dbname='leetcode', table_name='activity',create_table_sql=create_table_sql.replace("\n",";\n"))

Database 'leetcode' already exists.
Table 'activity' dropped successfully.
Table 'activity' created successfully.


In [3]:
PM.get_table_schema(dbname='leetcode', table_name='activity')

,column_name,data_type,character_maximum_length,numeric_precision,is_nullable
0,machine_id,integer,None,32.0,YES
1,process_id,integer,None,32.0,YES
2,activity_type,USER-DEFINED,None,NaN,YES
3,timestamp,numeric,None,NaN,YES


In [4]:
query = """
With Activity_ as  (
SELECT *, timestamp - LAG(timestamp) OVER (ORDER BY machine_id,process_id, activity_type) as processing_time
from Activity
)

Select  machine_id, ROUND(avg(processing_time),3) as processing_time
from Activity_
where activity_type ='end'
group by machine_id
"""

# Run the query and print the results
PM.run_query(dbname='leetcode', query=query)

,machine_id,processing_time
0,0,0.894
1,1,0.995
2,2,1.456
